In [1]:
# general packages
import numpy as np
import pandas as pd
import datatable as dt

In [4]:
%%time
data_datatable = dt.fread('features_processed.csv') 
data = data_datatable.to_pandas()
data = data.iloc[:,1:]

Wall time: 10.4 s


In [6]:
data

,Conditions,AreaNumber,PuLat,PuLong,create_by_hour,request_by_hour,pick,BookingSource_Internet,BookingSource_Permanent,BookingSource_StratumIVR,...,create_part_of_day_Afternoon,create_part_of_day_Evening,create_part_of_day_Morning,create_part_of_day_Night,request_part_of_day_Afternoon,request_part_of_day_Evening,request_part_of_day_Morning,request_part_of_day_Night,BookingisDispatch_False,BookingisDispatch_True
0,0,177,-33.921022,151.244202,10,11,False,False,False,False,...,False,False,True,False,False,False,True,False,False,True
1,0,111,-33.755931,151.273028,9,9,True,False,False,False,...,False,False,True,False,False,False,True,False,False,True
2,0,92,-33.897201,151.261205,0,0,False,True,False,False,...,False,False,False,True,False,False,False,True,False,True
3,0,182,-33.895716,151.228609,9,9,False,False,False,True,...,False,False,True,False,False,False,True,False,False,True
4,0,4,-33.859737,151.213307,21,21,False,True,False,False,...,False,False,False,True,False,False,False,True,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5999995,0,209,-33.960439,151.059493,8,9,True,False,False,False,...,False,False,True,False,False,False,True,False,False,True
5999996,0,401,-33.916820,151.202503,19,19,False,False,False,False,...,False,True,False,False,False,True,False,False,True,False
5999997,0,182,-33.891001,151.223461,15,15,False,False,False,False,...,True,False,False,False,True,False,False,False,False,True
5999998,0,19,-33.849802,151.211508,17,18,False,False,False,False,...,True,False,False,False,False,True,False,False,True,False


In [11]:
data.dtypes.head(10)

Conditions                    int64
AreaNumber                    int32
PuLat                       float64
PuLong                      float64
create_by_hour                int32
request_by_hour               int32
pick                           bool
BookingSource_Internet         bool
BookingSource_Permanent        bool
BookingSource_StratumIVR       bool
dtype: object

## Training

In [7]:
training_data = data.iloc[:3000000,:]
validation_data = data.iloc[3000000:,:]

In [8]:
y_train = training_data['TargetVariable']
X_train = training_data.drop(columns=['TargetVariable'])
y_val = validation_data['TargetVariable']
X_val = validation_data.drop(columns=['TargetVariable'])

KeyError: 'TargetVariable'